<a href="https://colab.research.google.com/github/P202U/Job-scrap/blob/main/Job_Scrap_0_2_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [27]:
keyword = 'fullstack'
location = 'Delhi'
pageNum = 0
job_url = f'https://www.linkedin.com/jobs/search?keywords={keyword}&location={location}&pageNum={pageNum}'

response = requests.get(job_url)
job_data = response.text
soup = BeautifulSoup(job_data, "lxml")
page_jobs = soup.find_all('div', {'class': 'base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card'})

In [28]:
job_id_list = []

page_jobs = soup.find_all('div', class_='job-search-card')

for job_card in page_jobs:
    try:
        urn_string = job_card.get("data-entity-urn")

        if urn_string:
            job_id = urn_string.split(":")[-1]

            print(job_id)
            job_id_list.append(job_id)

    except (AttributeError, IndexError) as e:
        print(f"Skipping a job card due to an error: {e}")
        continue

print("\nAll extracted Job IDs:", job_id_list)

4295430847
4284138568
4292182511
4267025199
4291646413
4288457079
4293604347
4273997962

All extracted Job IDs: ['4295430847', '4284138568', '4292182511', '4267025199', '4291646413', '4288457079', '4293604347', '4273997962']


In [32]:
job_list = []

for job_id in job_id_list:
    job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"

    try:
        job_response = requests.get(job_url)
        job_response.raise_for_status()
        job_soup = BeautifulSoup(job_response.text, "lxml")
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch job posting {job_id}: {e}")
        continue

    job_post = {
        "time_posted": None,
        "company_name": None,
        "job_title": None,
        "num_applicants": None,
        "job_id": None
    }

    job_post["job_id"] = job_id

    try:
        title_element = job_soup.select_one("h2.topcard__title")
        if title_element:
            job_post["job_title"] = title_element.get_text(strip=True)
    except Exception as e:
        print(f"Error extracting job title for {job_id}: {e}")

    try:
        company_element = job_soup.select_one("a.topcard__org-name-link")
        if company_element:
            job_post["company_name"] = company_element.get_text(strip=True)
    except Exception as e:
        print(f"Error extracting company name for {job_id}: {e}")

    try:
        time_element = job_soup.select_one("span.posted-time-ago__text")
        if time_element:
            job_post["time_posted"] = time_element.get_text(strip=True)
    except Exception as e:
        print(f"Error extracting posted time for {job_id}: {e}")

    try:
        applicants_selectors = [
            'figcaption.num-applicants__caption',
            'span.num-applicants__caption',
            'span.posted-time-ago__text'
        ]

        for selector in applicants_selectors:
            applicants_element = job_soup.select_one(selector)
            if applicants_element:
                text = applicants_element.get_text(strip=True)
                if "applicant" in text.lower():
                    job_post["num_applicants"] = text
                    break
    except Exception as e:
        print(f"Error extracting num applicants for {job_id}: {e}")

    job_list.append(job_post)
    print(f"Successfully scraped job {job_id}")

Successfully scraped job 4295430847
Successfully scraped job 4284138568
Successfully scraped job 4292182511
Successfully scraped job 4267025199
Successfully scraped job 4291646413
Successfully scraped job 4288457079
Successfully scraped job 4293604347
Successfully scraped job 4273997962


In [33]:
job_list

[{'time_posted': '11 hours ago',
  'company_name': 'U.S. Bank',
  'job_title': 'Full Stack Web Developer - JavaScript',
  'num_applicants': '73 applicants',
  'job_id': '4295430847'},
 {'time_posted': '7 minutes ago',
  'company_name': 'Golden Technology',
  'job_title': 'FullStack Developer',
  'num_applicants': '47 applicants',
  'job_id': '4284138568'},
 {'time_posted': '6 days ago',
  'company_name': 'Golden Technology',
  'job_title': 'Java Developer',
  'num_applicants': 'Over 200 applicants',
  'job_id': '4292182511'},
 {'time_posted': '1 month ago',
  'company_name': 'Nexaminds',
  'job_title': 'Senior Fullstack Developer (.Net / React)',
  'num_applicants': '38 applicants',
  'job_id': '4267025199'},
 {'time_posted': '1 week ago',
  'company_name': 'LeadStack Inc.',
  'job_title': 'Java Developer - 25-02904',
  'num_applicants': 'Over 200 applicants',
  'job_id': '4291646413'},
 {'time_posted': '6 days ago',
  'company_name': 'Aditi Consulting',
  'job_title': 'AI Engineer',
 

In [34]:
jobs_df = pd.DataFrame(job_list)
jobs_df

,time_posted,company_name,job_title,num_applicants,job_id
0,11 hours ago,U.S. Bank,Full Stack Web Developer - JavaScript,73 applicants,4295430847
1,7 minutes ago,Golden Technology,FullStack Developer,47 applicants,4284138568
2,6 days ago,Golden Technology,Java Developer,Over 200 applicants,4292182511
3,1 month ago,Nexaminds,Senior Fullstack Developer (.Net / React),38 applicants,4267025199
4,1 week ago,LeadStack Inc.,Java Developer - 25-02904,Over 200 applicants,4291646413
5,6 days ago,Aditi Consulting,AI Engineer,102 applicants,4288457079
6,22 hours ago,Jobot,Lead Software Engineer,Be among the first 25 applicants,4293604347
7,1 hour ago,Deloitte,Deloitte Technology Product Engineering Fullst...,55 applicants,4273997962
